In [2]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [3]:
half = sym.Rational(1,2)

p = RollRig.base_params(1)

In [8]:
# Generate Referecne Frame
wing_frame = mf.HomogenousTransform().R_x(p.q[0])#.R_y(p.alpha_r)


#Generate Mass Matrices
M_fwt = ele.MassMatrix(p.m_f,I_xx = p.I_xx_f)
M_w = ele.MassMatrix(p.m_w,I_xx = p.I_xx_w)


# Generate Rigid Elements
bar_wing = ele.RigidElement(wing_frame,M_w)

In [5]:
# Main Wing Aero Forces 
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,wing_frame.Translate(0,p.y_w,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = p.w_g,
                               rootAlpha = p.alpha_r,
                               alpha_zero = 0,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True).integrate((p.y_w,-p.s_w*half,p.s_w*half))

In [6]:
Forcing = ef.CustomForce(None) 

CompositeForce = ef.CompositeForce([wing_AeroForces,Forcing])
# create instance of the model
sm = mf.SymbolicModel.FromElementsAndForces(p,[bar_wing],ExtForces = CompositeForce)

In [7]:
sm.to_file('RollRigModel-Removed.py')